In [ ]:
#reading the data
import pandas as pd
df=pd.read_csv("/kaggle/input/breast-cancer-wisconsin-data/data.csv")
df.head()

In [ ]:
#shape of data frame
df.shape

In [ ]:
print('Canacer detected', round(df['diagnosis'].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('Malignant ', round(df['diagnosis'].value_counts()[1]/len(df) * 100,2), '% of the dataset')

The data set is  a little bit unbalanced but We can consider this ratio for model creation

In [ ]:
#drop Unnamed: 32
df1=df.drop(columns='Unnamed: 32')

In [ ]:
#there is no null values
print(df.info())
df1.diagnosis.value_counts()

There are no null values

In [ ]:
#Label Encoding to change the tha char value of diagnosis to 0s and 1
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df1['diagnosis']= label_encoder.fit_transform(df1['diagnosis'])
df1.diagnosis.value_counts()

0 or B : 212 

1 or M : 357

In [ ]:

#checking duplicates
print(df1.duplicated().sum())

In [ ]:
#Defining x and y
x=df1.iloc[:,2:]
y=df1.diagnosis


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y)

# Dimensionality reduction with PCA

In [ ]:
#getting the n_component for the PCA
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
#PCA is largely affected by scales and different features might have different scales.
#so we will standerise the data to get the components
stdSC=StandardScaler()
x_train_std=stdSC.fit_transform(x_train)
pcaModel=PCA()
pcaModel.fit(x_train_std)
pcaModel.explained_variance_
pcaModel.explained_variance_ratio_
#pcaModel.explained_variance_ratio_*100
np.cumsum(pcaModel.explained_variance_ratio_*100)


We can see that 24 componets define almost 99.9% of data .

In [ ]:
# create pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import  LogisticRegression
#from sklearn.linear_model import Ridge

logistic = LogisticRegression(max_iter=10000, tol=0.1)
steps_4_model=[('std',StandardScaler()),('pca',PCA(n_components=24)),('log',LogisticRegression())]
#parameters = [ {'model__alpha': np.arange(0, 0.2, 0.01) } ]


finalModel=Pipeline(steps=steps_4_model)
finalModel.fit(x_train,y_train)
finalModel.predict(x_test)
print("Train Score (Linear):",finalModel.score(x_train,y_train))
print("Test Score (Linear):",finalModel.score(x_test,y_test))


In [ ]:
print(pd.crosstab(y_train,finalModel.predict(x_train)))
print("**********************************************")
print(pd.crosstab(y_test,finalModel.predict(x_test)))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,finalModel.predict(x_test)))

In [ ]:
#to visulaise the new dimension

stdSc=StandardScaler()
stdSc.fit(x)
x_train_std=stdSc.transform(x_train)

pcaModel_24c=PCA(n_components=24)
pcaModel_24c.fit(x_train_std)
pcaModel_24c.explained_variance_
(pcaModel_24c.explained_variance_ratio_*100)
x_train_comp=pcaModel_24c.transform(x_train_std)
pd.DataFrame(x_train_comp).head()

In [ ]:
#checking the visualisation with 2 components of the new dimension
from matplotlib import pyplot as plt,style
A=pd.DataFrame(x_train_comp[:,0:2],index=x_train.index)

#pd.DataFrame[(A,columns=["c1","c2"])
A.columns=["c1","c2"]

import seaborn as sns


plt.figure(figsize=(14,7))
plt.subplot(1,2,1)
sns.scatterplot(data=x_train,x=x_train.radius_mean,y=x_train.texture_mean,hue=y_train).set_title('orig_datset')
plt.subplot(1,2,2)
sns.scatterplot(data=A,x=A.c1,y=A.c2,hue=df1.diagnosis).set_title('with PCA')


# Dimensionality reduction with LDA 

### The fitted model can also be used to reduce the dimensionality of the input by projecting it to the most discriminative directions, using the transform method.

In [ ]:
# create pipeline
from sklearn.discriminant_analysis import  LinearDiscriminantAnalysis
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state= 0)


logistic = LogisticRegression(max_iter=10000, tol=0.1)
steps_4_model=[('std',StandardScaler()),('lda', LinearDiscriminantAnalysis(n_components=1)),('log',LogisticRegression())]
#parameters = [ {'model__alpha': np.arange(0, 0.2, 0.01) } ]


finalModel=Pipeline(steps=steps_4_model)
finalModel.fit(x_train,y_train)
finalModel.predict(x_test)
print("Train Score (Linear):",finalModel.score(x_train,y_train))
print("Test Score (Linear):",finalModel.score(x_test,y_test))


In [ ]:
print(pd.crosstab(y_train,finalModel.predict(x_train)))
print(pd.crosstab(y_test,finalModel.predict(x_test)))

In [ ]:
#Visulaisation for LDA



import seaborn as sns
style.use('ggplot')

stdSc=StandardScaler()
stdSc.fit(x)
x_train_std=stdSc.transform(x_train)

LDAModel_24c=LinearDiscriminantAnalysis(n_components=1)
LDAModel_24c.fit(x_train_std,y_train)

x_train_lda=LDAModel_24c.transform(x_train_std)
pd.DataFrame(x_train_lda).head()



In [ ]:
trainLDA=pd.DataFrame(x_train_lda,columns=["c1"],index=x_train.index)
trainLDA['diagnosis']=y_train
trainLDA.head()

In [ ]:
plt.figure(figsize=(14,7))
plt.subplot(1,2,1)
sns.scatterplot(data=x_train,x=x_train.radius_mean,y=x_train.texture_mean,hue=y_train).set_title('orig_datset')
plt.subplot(1,2,2)
sns.scatterplot(data=trainLDA,x=trainLDA.c1,y=0,hue='diagnosis').set_title('with LDA')